In [2]:
import srsly
import pandas as pd

In [5]:
df = pd.read_csv("../outputs/bert_tiny/agnews_bert_tiny_2023-02-22T18-23-14/logs/train/instance_level.csv")

In [6]:
df

,logit_0,logit_1,logit_2,logit_3,unique_id,epoch
0,-0.016056,0.192554,-0.037239,0.107961,54006,0
1,0.033814,0.288741,-0.023788,0.141302,39827,0
2,0.104661,0.344355,-0.153033,0.488190,46042,0
3,-0.016797,0.491647,-0.106398,0.132242,98620,0
4,-0.015787,0.390886,-0.153998,0.125015,63191,0
...,...,...,...,...,...,...
167620,4.134343,0.174673,-1.270067,-2.196616,119213,279
167621,3.975576,-0.474066,-1.159083,-1.601688,99918,279
167622,-0.175468,3.906070,-2.184976,-1.478945,56202,279
167623,4.134739,-0.619041,-0.862050,-1.432592,14746,279
